In [ ]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.1'
spark_version = 'spark-3.3.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:2 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:3 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:5 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Hit:9 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:10 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:11 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Reading package lists... Done


In [ ]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [ ]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)

In [ ]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('home_sales')
df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [ ]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- date_built: string (nullable = true)
 |-- price: string (nullable = true)
 |-- bedrooms: string (nullable = true)
 |-- bathrooms: string (nullable = true)
 |-- sqft_living: string (nullable = true)
 |-- sqft_lot: string (nullable = true)
 |-- floors: string (nullable = true)
 |-- waterfront: string (nullable = true)
 |-- view: string (nullable = true)



In [ ]:
from pyspark.sql.types import IntegerType, DateType
df = df.withColumn('date', df['date'].cast(DateType()))
df = df.withColumn('date_built', df['date_built'].cast(IntegerType()))
df = df.withColumn('price', df['price'].cast(IntegerType()))
df = df.withColumn('bedrooms', df['bedrooms'].cast(IntegerType()))
df = df.withColumn('bathrooms', df['bathrooms'].cast(IntegerType()))
df = df.withColumn('sqft_living', df['sqft_living'].cast(IntegerType()))
df = df.withColumn('sqft_lot', df['sqft_lot'].cast(IntegerType()))
df = df.withColumn('floors', df['floors'].cast(IntegerType()))
df = df.withColumn('waterfront', df['waterfront'].cast(IntegerType()))
df = df.withColumn('view', df['view'].cast(IntegerType()))

In [ ]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: date (nullable = true)
 |-- date_built: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: integer (nullable = true)
 |-- sqft_living: integer (nullable = true)
 |-- sqft_lot: integer (nullable = true)
 |-- floors: integer (nullable = true)
 |-- waterfront: integer (nullable = true)
 |-- view: integer (nullable = true)



In [ ]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
from pyspark.sql.functions import round
df.filter(df['bedrooms'] == 4).\
      groupBy('date_built').avg().\
      select('date_built', round('avg(price)', 2)).\
      orderBy(df['date_built']).show()

+----------+--------------------+
|date_built|round(avg(price), 2)|
+----------+--------------------+
|      2010|           296800.75|
|      2011|            302141.9|
|      2012|           298233.42|
|      2013|           299999.39|
|      2014|           299073.89|
|      2015|           307908.86|
|      2016|           296050.24|
|      2017|           296576.69|
+----------+--------------------+



In [ ]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
df.filter((df['bedrooms'] == 3) & (df['bathrooms'] == 3)).\
      groupBy('date_built').avg().\
      select('date_built', round('avg(price)', 2)).\
      orderBy(df['date_built']).show()

+----------+--------------------+
|date_built|round(avg(price), 2)|
+----------+--------------------+
|      2010|           292859.62|
|      2011|           291117.47|
|      2012|           293683.19|
|      2013|           295962.27|
|      2014|           290852.27|
|      2015|            288770.3|
|      2016|           290555.07|
|      2017|           292676.79|
+----------+--------------------+



In [ ]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
df.filter((df['bedrooms'] == 3) & (df['bathrooms'] == 3) & (df['floors'] == 2) & (df['sqft_living'] >= 2000)).\
      groupBy('date_built').avg().\
      select('date_built', round('avg(price)', 2)).\
      orderBy(df['date_built']).show()

+----------+--------------------+
|date_built|round(avg(price), 2)|
+----------+--------------------+
|      2010|           285010.22|
|      2011|           276553.81|
|      2012|           307539.97|
|      2013|           303676.79|
|      2014|           298264.72|
|      2015|           297609.97|
|      2016|            293965.1|
|      2017|           280317.58|
+----------+--------------------+



In [ ]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

df.filter(df['price'] >= 350000).\
      groupBy('date_built').avg().\
      select('date_built', round('avg(price)', 2), 'avg(view)').\
      orderBy(df['date_built']).show()

print("--- %s seconds ---" % (time.time() - start_time))

+----------+--------------------+------------------+
|date_built|round(avg(price), 2)|         avg(view)|
+----------+--------------------+------------------+
|      2010|           463301.08|30.638909358879882|
|      2011|           472861.91|  32.2764405543399|
|      2012|           475524.98|33.484249084249086|
|      2013|           476837.88| 32.60732232591529|
|      2014|           475674.82|32.277142857142856|
|      2015|           471517.37|31.889278206026628|
|      2016|           478550.09| 32.90862944162436|
|      2017|           475884.72| 32.03489183531054|
+----------+--------------------+------------------+

--- 1.0394432544708252 seconds ---


In [ ]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [ ]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [ ]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

df.filter(df['price'] >= 350000).\
      groupBy('date_built').avg().\
      select('date_built', round('avg(price)', 2), 'avg(view)').\
      orderBy(df['date_built']).show()

print("--- %s seconds ---" % (time.time() - start_time))


+----------+--------------------+------------------+
|date_built|round(avg(price), 2)|         avg(view)|
+----------+--------------------+------------------+
|      2010|           463301.08|30.638909358879882|
|      2011|           472861.91|  32.2764405543399|
|      2012|           475524.98|33.484249084249086|
|      2013|           476837.88| 32.60732232591529|
|      2014|           475674.82|32.277142857142856|
|      2015|           471517.37|31.889278206026628|
|      2016|           478550.09| 32.90862944162436|
|      2017|           475884.72| 32.03489183531054|
+----------+--------------------+------------------+

--- 0.621443510055542 seconds ---


In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
df.write.partitionBy("date_built").mode("overwrite").parquet("sales_partitioned")

In [ ]:
# 11. Read the parquet formatted data.
p_df=spark.read.parquet('sales_partitioned')

In [ ]:
# 12. Create a temporary table for the parquet data.
p_df.createOrReplaceTempView('p_home_sales')

In [ ]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version.

start_time = time.time()

df.filter(df['price'] >= 350000).\
      groupBy('date_built').avg().\
      select('date_built', round('avg(price)', 2), 'avg(view)').\
      orderBy(df['date_built']).show()

print("--- %s seconds ---" % (time.time() - start_time))

+----------+--------------------+------------------+
|date_built|round(avg(price), 2)|         avg(view)|
+----------+--------------------+------------------+
|      2010|           463301.08|30.638909358879882|
|      2011|           472861.91|  32.2764405543399|
|      2012|           475524.98|33.484249084249086|
|      2013|           476837.88| 32.60732232591529|
|      2014|           475674.82|32.277142857142856|
|      2015|           471517.37|31.889278206026628|
|      2016|           478550.09| 32.90862944162436|
|      2017|           475884.72| 32.03489183531054|
+----------+--------------------+------------------+

--- 0.43067002296447754 seconds ---


In [ ]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [ ]:
# 15. Check if the home_sales is no longer cached
if spark.catalog.isCached("home_sales"):
  print("a table is till cached")
else:
  print("all clear")

all clear
